**Data EXTRACTION was done manually by downloading the CSV files from municipal open data portals.**

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('toronto_business_licenses.csv')

/var/folders/lm/g6mgw7vd5_d3xjgrm7k7tvz00000gn/T/ipykernel_41944/1940338114.py:1: DtypeWarning: Columns (7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('toronto_business_licenses.csv')


**Set option to suppress scientific notation on the phone numbers. Removing decimal places.**

In [3]:
pd.reset_option('display.float_format')

In [4]:
pd.set_option('display.float_format', lambda x: '%.f' % x)

**Rename columns.**

In [5]:
df = df.rename(columns={'Operating Name':'Business Name'
                        ,'Issued':'Issue Date'
                        ,'Cancel Date':'License Cancel Date'
                        ,'Licence No.': 'License Number'
                        ,'Business Phone':'Phone Number'
                        ,'Business Phone Ext.':'Phone Ext.'
                        ,'Business Phone Type':'Phone Type'
                       }
              )

**Add 2 new columns, 1 for phone # type and 1 for a concatenated address.**

In [6]:
df.insert(5,'Expiry Date',np.nan)
df.insert(8,'Phone Type',df['Phone Ext.'])
df.insert(9,'City','Toronto')
df['Address'] = df['Licence Address Line 1']+df['Licence Address Line 2']+df['Licence Address Line 3']

**Modify column dtypes to match their values more appriopriately.**

In [7]:
df['Issue Date'] = pd.to_datetime(df['Issue Date'])
df['Phone Ext.'] = pd.to_numeric(df['Phone Ext.'],errors='coerce')
df['Expiry Date'] = pd.to_datetime(df['Expiry Date'])

**Drop columns which are not needed.**

In [8]:
drop_cols = ['_id'
             ,'Client Name'
             ,'Conditions'
             ,'Free Form Conditions Line 1'
             ,'Free Form Conditions Line 2'
             ,'Plate No.'
             ,'Endorsements'
             ,'Licence Address Line 1'
             ,'Licence Address Line 2'
             ,'Licence Address Line 3'
             ,'License Cancel Date'
             ,'Last Record Update'
            ]
df = df.drop(drop_cols, axis=1)

**Lambda function to convert all Business Phone Type data to string. Then convert all missing values to np.nan and all non null values to title case.**

In [9]:
df['Phone Type'] = df['Phone Type'].apply(lambda x: str(x))

for i in df.index:
    if df['Phone Type'].iloc[i] != 'nan':
        if df['Phone Type'].iloc[i].isnumeric():
#             print(df['Business Phone Type'].iloc[i])
            df.at[i,'Phone Type'] = 'nan'

for i in df.index:
    if df['Phone Type'].iloc[i] == 'nan':
        df.at[i,'Phone Type'] = np.nan

df['Phone Type'] = df['Phone Type'].str.title()

**Function to reformat phone numbers so they are more human readable.**

In [10]:
def format_number(x):
    NPA = str(x)[:3]
    NXX = str(x)[3:6]
    XXXX = str(int(x))[6:]

    formatted_number = '(' + NPA + ')' + ' ' + NXX + ' - ' + XXXX
    return formatted_number

df['Phone Number'] = df['Phone Number'].apply(lambda x: format_number(x) if pd.isnull(x) == False else x)

In [11]:
# df[(df["Business Phone"].notnull()) | (df["Business Phone Type"].notna()) | (df["Business Phone Ext."].notna())]

In [12]:
columns = df.columns
print(columns)
print(len(columns))

Index(['Category', 'License Number', 'Business Name', 'Issue Date',
       'Expiry Date', 'Phone Number', 'Phone Type', 'City', 'Phone Ext.',
       'Address'],
      dtype='object')
10


**Reorder columns in dataframe. I arbitrarily decided on column ordering based on a gut feeling of what is most intuitive.**

In [13]:
df = df[
    [columns[1],columns[2],columns[0],columns[3],columns[4],columns[5],columns[6],columns[8],columns[9],columns[7]]
]

In [14]:
df

,License Number,Business Name,Category,Issue Date,Expiry Date,Phone Number,Phone Type,Phone Ext.,Address,City
0,B02-4741962,TAXIFY,PRIVATE TRANSPORTATION COMPANY,2018-01-18,NaT,NaN,NaN,NaN,"35 OAK ST, #304TORONTO, ONM9N 1A1",Toronto
1,B02-4728645,INSTARYDE,PRIVATE TRANSPORTATION COMPANY,2017-09-08,NaT,NaN,NaN,NaN,"425 ALNESS STTORONTO, ONM3J 2T8",Toronto
2,B02-4807677,DRIVEHER TRANSPORTATION,PRIVATE TRANSPORTATION COMPANY,2018-03-08,NaT,NaN,NaN,NaN,"100 DUNDAS ST E, #502MISSISSAUGA, ONL5A 1W5",Toronto
3,B02-4653451,UBER CANADA,PRIVATE TRANSPORTATION COMPANY,2016-08-16,NaT,(514) 589 - 8345,NaN,NaN,"121 BLOOR ST E, #1600TORONTO, ONM4W 3M5",Toronto
4,B02-4691773,FACEDRIVE,PRIVATE TRANSPORTATION COMPANY,2017-04-04,NaT,(289) 452 - 0164,NaN,NaN,"44 EAST BEAVER CREEK RD, #16RICHMOND HILL, ONL...",Toronto
...,...,...,...,...,...,...,...,...,...,...
154223,V27-5234313,NaN,NON-MOTORIZED REFRESHMENT VEHICLE OWNER,2022-10-26,NaT,NaN,NaN,NaN,NaN,Toronto
154224,V27-5233581,AMSTERDAM FRIES,NON-MOTORIZED REFRESHMENT VEHICLE OWNER,2022-10-27,NaT,NaN,NaN,NaN,"1355 GERRARD ST ETORONTO, ONM4L 1Y8",Toronto
154225,V27-5236191,SUN SET HUT CARIBBEAN,NON-MOTORIZED REFRESHMENT VEHICLE OWNER,2022-11-07,NaT,NaN,NaN,NaN,"3480 MCNICOLL AVE, #10TORONTO, ONM1V 5A7",Toronto
154226,V27-5238632,NaN,NON-MOTORIZED REFRESHMENT VEHICLE OWNER,2022-11-22,NaT,NaN,NaN,NaN,NaN,Toronto


**Convert dataframe back to CSV for concatenation.**

In [15]:
 df.to_csv('/Users/graemebalint/Documents/Python/Jupyter Notebooks/Canadian Businesses/csv_reformatted/toronto_reformatted.csv',index=False)  